In [2]:
import os

In [3]:
import mysql.connector
from mysql.connector import Error
# Establish connection
mydb = mysql.connector.connect(
   host="hostname",
    user="username",
    password="<password>",
)

# Check if connection was successful
if mydb.is_connected():
  print("Connected to MySQL!")
    
else:
  print("Failed to connect to MySQL")

Connected to MySQL!


In [4]:
# Specify the name of the new database to create
db_name = "youtubecommunity"

# Create a cursor object to execute SQL queries
mycursor = mydb.cursor()

# Execute the SQL query to create a new database
mycursor.execute(f"CREATE DATABASE {db_name}")

print(f"Database '{db_name}' created successfully!")


DatabaseError: 1007 (HY000): Can't create database 'youtubecommunity'; database exists

In [5]:
# show databases :-

# Create a cursor object to execute SQL queries
mycursor = mydb.cursor()

# Execute the SQL query to create a new database
mycursor.execute("SHOW DATABASES")

for db in mycursor:
    print(db)


('demo',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('youtubecommunity',)


In [6]:
# create table

mydb = mysql.connector.connect(
    host="hostname",
    user="username",
    password="<password>",
    database_name="<database_name>"
)
mycursor = mydb.cursor()

mycursor.execute("CREATE TABLE students (name VARCHAR(255), age INTEGER(10))")


ProgrammingError: 1050 (42S01): Table 'students' already exists

In [7]:
# show tables 

mycursor.execute("SHOW TABLES")

for tb in mycursor:
    print(tb)

('students',)


In [7]:
# add data in tables

sqlFormula = "INSERT INTO students (name,age) VALUES (%s,%s)"
students =  [("Bob",22),
             ("Aman",23),
             ("Rahul",24),
             ("Jacob",25),
             ("Michalle",27)]

mycursor.executemany(sqlFormula,students)
mydb.commit()

In [8]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import json
from typing import Any

class mysql_operation:
    def __init__(self, host: str, user: str, password: str, database_name: str):
        self.host = host
        self.user = user
        self.password = password
        self.database_name = database_name
    
    def create_connection(self):
        """Create a MySQL connection."""
        try:
            connection = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database_name
            )
            if connection.is_connected():
                print("Connected to MySQL database")
            return connection
        except Error as e:
            print(f"Error while connecting to MySQL: {e}")
            return None

    def create_database(self):
        """Create a MySQL database if it doesn't exist."""
        connection = mysql.connector.connect(
            host=self.host,
            user=self.user,
            password=self.password
        )
        cursor = connection.cursor()
        try:
            cursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.database_name}")
            print(f"Database '{self.database_name}' created or already exists.")
            return self.database_name
        except Error as e:
            print(f"Error creating database: {e}")
        finally:
            cursor.close()
            connection.close()

    def create_table(self, create_table_sql: str):
        """Create a table in the MySQL database using a SQL query."""
        connection = self.create_connection()
        cursor = connection.cursor()
        try:
            cursor.execute(create_table_sql)
            print("Table created successfully.")
            return True
        except Error as e:
            print(f"Error creating table: {e}")
            return None 
        finally:
            cursor.close()
            connection.close()

    def insert_record(self, record: dict, table_name: str):
        """Insert one or many records into the specified MySQL table."""
        connection = self.create_connection()
        cursor = connection.cursor()

        if isinstance(record, list):
            # Ensure all items in the list are dictionaries
            for data in record:
                if not isinstance(data, dict):
                    raise TypeError("Each record in the list must be a dictionary")

            # Prepare SQL for inserting multiple records
            columns = ', '.join(record[0].keys())
            placeholders = ', '.join(['%s'] * len(record[0]))
            sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

            # Extract the values for each record as tuples
            values = [tuple(data.values()) for data in record]
            cursor.executemany(sql, values)
            
        elif isinstance(record, dict):
            # Prepare SQL for inserting a single record
            columns = ', '.join(record.keys())
            placeholders = ', '.join(['%s'] * len(record))
            sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

            # Extract the values as a tuple
            values = tuple(record.values())
            cursor.execute(sql, values)
        else:
            raise TypeError("Record must be a dictionary or a list of dictionaries")

        # Commit the transaction
        connection.commit()
        print(f"Record(s) inserted into {table_name}.")
        
        cursor.close()
        connection.close()


    def bulk_insert(self, datafile: str, table_name: str, unique_field: str = None):
        """Bulk insert records from a CSV or Excel file."""
        connection = self.create_connection()
        cursor = connection.cursor()

        if datafile.endswith('.csv'):
            data = pd.read_csv(datafile)
        elif datafile.endswith('.xlsx'):
            data = pd.read_excel(datafile)
        
        data_json = json.loads(data.to_json(orient='records'))

        for record in data_json:
            columns = ', '.join(record.keys())
            placeholders = ', '.join(['%s'] * len(record))
            if unique_field:
                select_query = f"SELECT COUNT(*) FROM {table_name} WHERE {unique_field} = %s"
                cursor.execute(select_query, (record[unique_field],))
                result = cursor.fetchone()
                if result[0] == 0:
                    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
                    cursor.execute(sql, tuple(record.values()))
                else:
                    print(f"Record with {unique_field}={record[unique_field]} already exists. Skipping insertion.")
            else:
                sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
                cursor.execute(sql, tuple(record.values()))
        
        connection.commit()
        print("Bulk insert completed.")
        cursor.close()
        connection.close()

    def find(self, query: dict = {}, table_name: str = None):
        """Retrieve records from the specified MySQL table based on the query."""
        connection = self.create_connection()
        cursor = connection.cursor(dictionary=True)  # This will return results as dictionaries

        # If no query is specified, fetch all records
        if not query:
            sql = f"SELECT * FROM {table_name}"
            cursor.execute(sql)
        else:
            # Construct the WHERE clause based on the query dictionary
            conditions = ' AND '.join([f"{key} = %s" for key in query.keys()])
            sql = f"SELECT * FROM {table_name} WHERE {conditions}"
            cursor.execute(sql, tuple(query.values()))

        # Fetch all the results
        results = cursor.fetchall()

        cursor.close()
        connection.close()

        return results

    def update(self, table_name: str, query: dict, new_values: dict):
        """Update records in the MySQL table based on the query and new values."""
        connection = self.create_connection()
        cursor = connection.cursor()

        try:
            # Construct the SET clause from the new_values dictionary
            set_clause = ', '.join([f"{key} = %s" for key in new_values.keys()])

            # Construct the WHERE clause from the query dictionary
            where_clause = ' AND '.join([f"{key} = %s" for key in query.keys()])

            # Prepare the SQL UPDATE statement
            sql = f"UPDATE {table_name} SET {set_clause} WHERE {where_clause}"

            # Combine the values for SET and WHERE clauses
            values = tuple(new_values.values()) + tuple(query.values())

            # Execute the SQL statement
            cursor.execute(sql, values)
            connection.commit()

            print(f"Record(s) updated in {table_name} where {query}.")
            
        except mysql.Error as e:  # Make sure to replace `Error` with the relevant exception class
            print(f"Error updating record: {e}")
        
        finally:
            cursor.close()
            connection.close()


    def delete(self, table_name: str, query: dict):
        """Delete records from the MySQL table based on the query."""
        connection = self.create_connection()
        cursor = connection.cursor()

        try:
            # Construct the WHERE clause from the query dictionary
            where_clause = ' AND '.join([f"{key} = %s" for key in query.keys()])

            # Prepare the DELETE statement
            sql = f"DELETE FROM {table_name} WHERE {where_clause}"

            # Combine the values for the WHERE clause
            values = tuple(query.values())

            # Execute the SQL statement
            cursor.execute(sql, values)
            connection.commit()

            print(f"Record(s) deleted from {table_name} where {query}.")
        
        except mysql.Error as e:  # Use the appropriate MySQL error handling class
            print(f"Error deleting record: {e}")
        
        finally:
            cursor.close()
            connection.close()



In [9]:
# Creating an instance of mysql_operation
mysql_connector = mysql_operation(
    host="hostname",
    user="username",
    password="<password>",
    database_name="<database_name>"
)

In [29]:
# Example usage of one insert_record
record = {
    'name': 'John Doe',
    'age': '50'
}
mysql_connector.insert_record(table_name="students", record=record)


Connected to MySQL database
Record(s) inserted into students.


In [30]:
# Example usage of many insert_record
record =[
    {'name': 'Abhisek','age': '55'},
    {'name': 'Amitabh','age':'81'},
    {'name':'Ashwaria','age':'50'}    
    ]
mysql_connector.insert_record(table_name="students", record=record)


Connected to MySQL database
Record(s) inserted into students.


In [32]:
# Find all records in the 'students' table
records = mysql_connector.find(table_name="students")
print(records)


Connected to MySQL database
[{'name': 'Rachel', 'age': 22}, {'name': 'Rachel', 'age': 22}, {'name': 'Bob', 'age': 22}, {'name': 'Aman', 'age': 23}, {'name': 'Rahul', 'age': 24}, {'name': 'Jacob', 'age': 25}, {'name': 'Michalle', 'age': 27}, {'name': 'Bob', 'age': 22}, {'name': 'Aman', 'age': 23}, {'name': 'Rahul', 'age': 24}, {'name': 'Jacob', 'age': 25}, {'name': 'Michalle', 'age': 27}, {'name': 'John Doe', 'age': 50}, {'name': 'John Doe', 'age': 50}, {'name': 'Abhisek', 'age': 55}, {'name': 'Amitabh', 'age': 81}, {'name': 'Ashwaria', 'age': 50}, {'name': 'John Doe', 'age': 50}, {'name': 'Abhisek', 'age': 55}, {'name': 'Amitabh', 'age': 81}, {'name': 'Ashwaria', 'age': 50}]


In [33]:
# Find records where the 'name' is 'Alice' and 'age' is 30
query = {'name': 'Amitabh', 'age': 81}
records = mysql_connector.find(query=query, table_name="students")
print(records)


Connected to MySQL database
[{'name': 'Amitabh', 'age': 81}, {'name': 'Amitabh', 'age': 81}]


In [11]:
# Update the 'age' of the student whose 'name' is 'Alice'
query = {'name': 'Amitabh'}
new_values = {'age': 31}

mysql_connector.update(table_name="students", query=query, new_values=new_values)


Connected to MySQL database
Record(s) updated in students where {'name': 'Amitabh'}.


In [23]:
query = {'name': "John Doe", 'age': 50}
mysql_connector.delete(table_name='students', query=query)


Connected to MySQL database
Record(s) deleted from students where {'name': 'John Doe', 'age': 50}.


In [5]:
mysql_connector.create_database()

Database 'youtubecommunity' created or already exists.


'youtubecommunity'

In [7]:
mysql_connector.create_table('students')

Connected to MySQL database
Error creating table: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'students' at line 1


In [10]:
query = {'name': 'Amitabh','age':31}
mysql_connector.delete(query=query,table_name='students')

Connected to MySQL database
Record(s) deleted from students where {'name': 'Amitabh', 'age': 31}.


In [14]:
datafile = '/home/seaflux/Documents/mysqlconnectorpkg/experiments/students.csv'
mysql_connector.bulk_insert(datafile=datafile, table_name='students', unique_field='name')


Connected to MySQL database
Bulk insert completed.


In [24]:
def add(a: int, b: int) -> int:
    return a + b

In [25]:
add(2.5,3)

5.5

In [26]:
from ensure import ensure_annotations

@ensure_annotations
def add(a: int, b: int) -> int:
    return a + b

# If you call `add("2", "3")`, it will raise a TypeError because the arguments don't match the expected types (int).


In [27]:
add(2.5,3)

EnsureError: Argument a of type <class 'float'> to <function add at 0x7fd58e58cae0> does not match annotation type <class 'int'>